In [61]:
import math
import os
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

stopwords = set(stopwords.words('English'))

In [62]:
wnl = WordNetLemmatizer()

def cleanse(text):
    text = ''.join([ x for x in str(text).lower() if x in string.printable ])
    text = re.sub(r'\b\d+\b', '', text)
    text = text.replace('\n', ' ')
    text = re.sub(r'[#`\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;$,?\'%]', ' ', text)
    text = re.sub('\s+', ' ', text)
    tokens = nltk.word_tokenize(text)
    tokens = [ abbrev[t] if t in abbrev else t for t in tokens ]
    tokens = [ wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i, j in pos_tag(tokens) ]
    #text = ' '.join(t for t in tokens if t not in stopwords)
    text = ' '.join(tokens)
    return text

path = 'c:/Users/bill/Documents/projects/data/chatbot'
data = pd.read_csv(os.path.join(path, 'sample'), sep='\t', names=[
    'rpt_mnth', 'cnv_id', 'msg_id', 'agent', 'msg_type', 'channel', 
    'creat_ts', 'text', 'intent', 'score', 'resp_cd'
])
data = data[data['agent'].isin(['coremobile', 'ccp'])]['text']\
    .apply(lambda x : cleanse(x)).str.split('.').explode()\
    .reset_index(drop=True)

In [63]:
def masking(text):
    text = re.sub(r'\{+(\s+)?\{+[A-Za-z0-9\s]+\}+(\s+)?\}+', '[URL]', text)
    tokens = str(text).split()
    keys = [ t for t in tokens if t not in stopwords ]
    out = []
    if len(keys) >= 4:
        size = min(max(math.ceil(len(keys) * 0.4), 1), 5)
        masks = np.random.choice(keys, replace=False, size=size)
        for mask in masks:
            s = tokens.copy()
            s[s.index(mask)] = '[MASK]'
            s = ' '.join(s).strip() + '\t' + mask
            if '{ { link } }' not in s:
                out.append(s)
    return out
data = data.apply(lambda x : masking(x)).to_frame()
data = data['text'].apply(pd.Series).unstack().reset_index(drop=True).dropna()

In [64]:
with open('MLM.txt', 'w') as f:
    f.write('text\tmask\n')
    for d in data.values:
        f.write(d + '\n')

In [65]:
vocab = set()
for r in data.str.split():
    vocab.update(r)

In [66]:
len(vocab)

32101